## Libraries

In [196]:
# libraries

import numpy as np
import pandas as pd
from decimal import *
getcontext().prec = 17 # for double precision in saved files

# This notebook generates STA, OBS, and SRC files for tomography. 

# The STA file is simple and was created with excel - where 
# it is easily modified - but is imported here and converted to ascii.

## Sample notes and two particular inputs

In [197]:
sample = 'Berea_TS7' # 'smcva13' # 'Berea_TS7'
# sample properties
Vp = 3.570 # km/s # 3.106 for saied 3.151 benchtop bts7 3570 pressurized bts7
Diameter = .99*25.4 # mm # 1.0 for saied  berea7s7=0.99
Length = (1.985)*25.4 # mm # 2.02 for saied  bereats7=1.985

# inputs

# choose dimensions (for all files)
dimensions = 'mm' # m or mm

# absolute or relative times
absolute = False

## Stations File

In [198]:
# 1st row: nsta 0 0 0 0 0 0
# 2nd+ rows: id_sta x_sta y_sta z_sta dtp_sta (=0) dts_sta (=0) name

# load from excel spreadsheet that has all the calculations
df_sta = pd.read_excel("fsta.xlsx",header=None,sheet_name=dimensions)

# save as ascii
df_sta.iloc[:,:-2].to_csv("fsta_RGL.asc",sep='\t',index=False,header=False,encoding='ascii')
df_sta
# edit data types in first row
with open('fsta_RGL.asc','r') as file:
    data = file.readlines()
firstrowsta = data[0].split()
firstrowsta = [x.split('.')[0] for x in firstrowsta]
print('before:',data[0:2],'\n')
data[0] = ('\t'.join(str(int(x)) for x in firstrowsta))+'\n'
print('after:',data[0:2])
# re-save
with open('fsta_RGL_'+dimensions+'.asc','w') as file:
    file.writelines(data)

before: ['10\t0.0\t0.0\t0.0\t0\t0\t0\n', '1\t8.254999999999999\t0.0\t0.0\t0\t0\tOZZB\n'] 

after: ['10\t0\t0\t0\t0\t0\t0\n', '1\t8.254999999999999\t0.0\t0.0\t0\t0\tOZZB\n']


# Load picks and locations into dataframe for OBS and SRC

In [199]:
file_path = "datasets/a_python_processed/"+sample+"_df_TOMO_vel3570.csv"

df = pd.read_csv(file_path)
df.tail()

,evnt_index,file_name,trigger_time,trigger_time_zeroed,diff_trig,pulse_flag,sen1,sen2,sen3,sen4,sen5,sen6,sen7,sen8,sen9,sen10,t_E,x_E,y_E,z_E
24101,98052,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98245.asd,47066.685499,1784.685499,0.008840,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.4,-2.4,NaN,NaN,NaN,NaN
24102,98055,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98248.asd,47066.883819,1784.883819,0.198308,0,NaN,NaN,NaN,3.2,6.2,NaN,1.8,-0.2,-1.2,-3.6,NaN,NaN,NaN,NaN
24103,98056,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98249.asd,47067.334586,1785.334586,0.450766,0,11.0,NaN,NaN,0.0,0.0,NaN,-1.4,-3.6,-1.4,-1.6,NaN,NaN,NaN,NaN
24104,98060,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98252.asd,47067.423580,1785.423580,0.088986,0,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN,-1.4,NaN,NaN,NaN,NaN,NaN
24105,98061,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98253.asd,47067.651867,1785.651867,0.228288,0,-2.4,NaN,-3.0,-6.4,-5.4,-6.2,NaN,-4.0,NaN,NaN,-3.174765,5.481244,6.864838,29.561184


## Observations File

In [200]:
# 1st row: nt nP (=nt) nS (=0) 0 0 0

# remove non-source data
dfmask = ~df.isna()
src = df[dfmask['t_E']==True].reset_index()
print('sources:')
display(src.tail(2))

# compute nt (number of observations)
srcmask = ~src.isna()
senlist = ['sen'+str(x+1) for x in range(10)]
obs = src[srcmask[senlist]==True]
obs = obs[senlist]
obsmask = ~obs.isna()
print('\nobservations mask:')
display(obs.tail(2))
nt = int(obsmask.sum().sum())
nP = nt
nS = int(0)
firstrowobs = np.array([nt,nP,nS,0,0,0])
print('\nfirst row of f_obs:',firstrowobs)

sources:


,index,evnt_index,file_name,trigger_time,trigger_time_zeroed,diff_trig,pulse_flag,sen1,sen2,sen3,...,sen5,sen6,sen7,sen8,sen9,sen10,t_E,x_E,y_E,z_E
2895,24100,98043,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98237.asd,47066.676648,1784.676648,0.038398,0,0.4,0.6,-1.4,...,-2.6,-2.4,2.4,-2.2,-0.6,-0.2,-3.817115,4.524645,5.450537,28.279921
2896,24105,98061,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_98253.asd,47067.651867,1785.651867,0.228288,0,-2.4,NaN,-3.0,...,-5.4,-6.2,NaN,-4.0,NaN,NaN,-3.174765,5.481244,6.864838,29.561184



observations mask:


,sen1,sen2,sen3,sen4,sen5,sen6,sen7,sen8,sen9,sen10
2895,0.4,0.6,-1.4,-2.6,-2.6,-2.4,2.4,-2.2,-0.6,-0.2
2896,-2.4,NaN,-3.0,-6.4,-5.4,-6.2,NaN,-4.0,NaN,NaN



first row of f_obs: [22824 22824     0     0     0     0]


In [201]:
# 2nd+ rows: id_obs t_obs dt_obs id_src id_sta id_ray (=0)

# gather all t_obs
if absolute:
    obs = (obs*1e-6).add(src['trigger_time_zeroed'], axis=0)
    # constant error
    dt_obs = np.ones(nt)*2e-7
else:
    obs = np.around(obs, decimals=1)
    dt_obs = np.ones(nt)*0.2
    
t_obs = obs.values.flatten()

# compute ids
obslist = [x+1 for x in range(nt)]
id_obs = np.array(obslist).astype(int)
#
stalist = [x%10+1 for x in range(len(t_obs))]
id_sta = np.array(stalist).astype(int)
#
srcdict = src['evnt_index'].to_dict()
srclist = [srcdict[x//10] for x in range(len(t_obs))]
id_src = np.array(srclist).astype(int)

# dataframe
df_obs = pd.DataFrame({"0":id_obs,
                      "1":t_obs[~np.isnan(t_obs)],
                      "2":dt_obs,
                      "3":id_src[~np.isnan(t_obs)],
                      "4":id_sta[~np.isnan(t_obs)],
                      "5":np.zeros(nt).astype(int)})
# insert first row
df_first = pd.DataFrame([firstrowobs],columns=['0','1','2','3','4','5']) 
df_obs = pd.concat([df_first,df_obs]).reset_index(drop=True)

# save (data types of firstrow are lost)
df_obs.to_csv("fobs_RGL.asc",sep='\t',index=False,header=False,
              encoding='ascii')
df_obs.head()
# edit data types in first row
with open('fobs_RGL.asc','r') as file:
    data = file.readlines()
print('before:',data[0:2],'\n')
data[0] = ('\t'.join(str(x) for x in firstrowobs))+'\n'
print('after:',data[0:2])
# re-save
if absolute:
    with open('fobs_RGL_absolute.asc','w') as file:
        file.writelines(data)
else:      
    with open('fobs_RGL_relative.asc','w') as file:
        file.writelines(data)   

before: ['22824\t22824.0\t0.0\t0\t0\t0\n', '1\t7.4\t0.2\t28\t1\t0\n'] 

after: ['22824\t22824\t0\t0\t0\t0\n', '1\t7.4\t0.2\t28\t1\t0\n']


## Sources File

In [202]:
# 1st row: nsrc neqks nshot (=0) nblast (=0) neqks_out (=0) 1 1 1

nsrc = len(src)
neqks = nsrc
nshot = 0
nblast = 0
neqks_out = int(0)

firstrowsrc = np.array([nsrc,neqks,nshot,nblast,neqks_out,1,1,1])
print('\nfirst row of f_src:',firstrowsrc)


first row of f_src: [2897 2897    0    0    0    1    1    1]


In [203]:
# 2nd+ rows: x_src y_src z_src t0_src kp_src kt_src in_src id_srcs

# gather all sources
locations = ['x_E','y_E','z_E']
df_events = src[locations].copy()
if absolute:
    df_events['t_E'] = (src['t_E']*1e-6).add(src['trigger_time_zeroed'], axis=0)
else:
    df_events['t_E'] = src['t_E']
display(df_events.tail())

# initialize flags
kplist = [x+1 for x in range(nsrc)]
kp_src = np.array(kplist).astype(int)
kt_src = kp_src
in_src = np.ones(nsrc).astype(int)
id_events = src['evnt_index'].values.astype(int)

# dataframe
if dimensions=='m':
    df_src = pd.DataFrame({"0":df_events['x_E']/1000,
                      "1":df_events['y_E']/1000,
                      "2":df_events['z_E']/1000,
                      "3":df_events['t_E'],
                      "4":kp_src,
                      "5":kt_src,
                     "6":in_src,
                     "7":id_events})
else:
    df_src = pd.DataFrame({"0":df_events['x_E'],
                      "1":df_events['y_E'],
                      "2":df_events['z_E'],
                      "3":df_events['t_E'],
                      "4":kp_src,
                      "5":kt_src,
                     "6":in_src,
                     "7":id_events})        
        
# insert first row
df_1st = pd.DataFrame([firstrowsrc],columns=['0','1','2','3','4','5','6','7']) 
df_src = pd.concat([df_1st,df_src]).reset_index(drop=True)

# save
df_src.to_csv("fsrc_RGL.asc",sep='\t',index=False,header=False,encoding='ascii')
df_src.head()
# edit data types in first row
with open('fsrc_RGL.asc','r') as file:
    data = file.readlines()
print('before:',data[0:2],'\n')
data[0] = ('\t'.join(str(x) for x in firstrowsrc))+'\n'
print('after:',data[0:2])
# re-save
if absolute:
    with open('fsrc_RGL_'+dimensions+'_absolute.asc','w') as file:
        file.writelines(data)
else:      
    with open('fsrc_RGL_'+dimensions+'_relative.asc','w') as file:
        file.writelines(data)

,x_E,y_E,z_E,t_E
2892,9.059403,-3.513611,40.498485,-1.507051
2893,-9.834153,-2.185611,23.522287,-2.719888
2894,6.560728,1.711875,4.960493,-1.173373
2895,4.524645,5.450537,28.279921,-3.817115
2896,5.481244,6.864838,29.561184,-3.174765


before: ['2897.0\t2897.0\t0.0\t0.0\t0\t1\t1\t1\n', '-3.943572979311522\t-11.357437613496545\t23.844035960587238\t-0.15437776252422605\t1\t1\t1\t28\n'] 

after: ['2897\t2897\t0\t0\t0\t1\t1\t1\n', '-3.943572979311522\t-11.357437613496545\t23.844035960587238\t-0.15437776252422605\t1\t1\t1\t28\n']


## Event reference file for relative times

In [204]:
TforAE = src[['evnt_index','trigger_time_zeroed']]

# save (data types of firstrow are lost)
TforAE.to_csv("TforAE_RGL.asc",sep='\t',index=False,header=False,
              encoding='ascii')
TforAE.head()

,evnt_index,trigger_time_zeroed
0,28,892.612114
1,50,990.691080
2,86,1006.023389
3,89,1008.443979
4,2319,1024.994269


In [210]:
# num is id_obs from Jean indicating t_E > some arrival times...?
num = 10080
event = df_obs.iloc[num,:][3]
piezo = df_obs.iloc[num,:][4]
print('evnt_index =',int(event),'detected by sen{}'.format(int(piezo)))
col = ['file_name']+senlist
df[df['evnt_index']==event][col]

evnt_index = 38915 detected by sen7


,file_name,sen1,sen2,sen3,sen4,sen5,sen6,sen7,sen8,sen9,sen10
12173,C:/Downloads/TRANSFER/Berea_TS7_5MPa_0_44969.asd,-6.2,NaN,-6.0,6.6,-9.6,NaN,-19.8,1.0,-9.8,-6.2
